In [25]:
from langchain.chat_models import ChatOllama
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OllamaEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOllama(
    model="mistral:latest",
    temperature=0.1,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

loader = CSVLoader("./file/data.csv")
docs = loader.load_and_split()

embeddings = OllamaEmbeddings(
    model="mistral:latest"
)

cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorStore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorStore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a data analyst. Answer questions using only the given context. If you don't know the answer, just say you don't know. Do NOT make it up. And also, do not talk about anything else:\n{context}"),
    ("human", "{question}")
])

chain = {
    "context": retriever,
    "question": RunnablePassthrough()
    } | prompt | llm

chain.invoke("Tell me about the data. Answer in Korean.")

 저기에는 CSV 파일 './file/data.csv'에서 추출한 데이터 4개가 있습니다. 각 데이터는 "InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country" 라는 필드로 구성되어 있습니다. 단순히 데이터의 존재에 대해 설명하고 있습니다. 데이터에서 특정 값을 추출하려면 더욱 자세한 질문을 주시면 답변하도록 할게요. (There are four pieces of data extracted from the CSV file './file/data.csv'. Each data consists of fields "InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country". I'm just explaining that these data exist. If you want to extract specific values from the data, please ask more specifically.)

AIMessage(content=' 저기에는 CSV 파일 \'./file/data.csv\'에서 추출한 데이터 4개가 있습니다. 각 데이터는 "InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country" 라는 필드로 구성되어 있습니다. 단순히 데이터의 존재에 대해 설명하고 있습니다. 데이터에서 특정 값을 추출하려면 더욱 자세한 질문을 주시면 답변하도록 할게요. (There are four pieces of data extracted from the CSV file \'./file/data.csv\'. Each data consists of fields "InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country". I\'m just explaining that these data exist. If you want to extract specific values from the data, please ask more specifically.)')